In [1]:
import math
import numpy as np
import csv
import statistics

# initialize random number generator
rng = np.random.default_rng(0)

# numbers of arms, rounds
N = 5
T = 1000

# number of times to run each randomized algorithm, and lists for storing total payouts from each run
K = 5
simple_payouts = []
exp3_payouts = []
ts_payouts = []

# lists of 1000 payouts (losses) for each arm
payouts = [ [] for i in range(N) ]

# read in arm payouts from csv file
with open('payouts.csv', newline='') as csvfile:
	csv_reader = csv.reader(csvfile, delimiter=',')
	next(csv_reader) # skip row with column headers
	# fill out lists with payouts
	for row in csv_reader:
		payouts[0].append(int(row[0]))
		payouts[1].append(int(row[1]))
		payouts[2].append(int(row[2]))
		payouts[3].append(int(row[3]))
		payouts[4].append(int(row[4]))

In [2]:
# Simple MAB algorithm with MWU as chosen full-information algorithm A

# probability of exploring
delta = 0.3

# pick a practical value for epsilon for MWU
epsilon = np.sqrt(np.log(N) / (T))

for k in range(K):

	# keep track of cumulative loss for Simple MAB
	simple_payout = 0

	# initial vector of arm selection probabilities
	x = [(1/N) for i in range(N)]

	# vector of weights for MWU
	weights = [1 for i in range(N)]

	# algorithm
	for t in range(T):
		# choose whether to explore or exploit
		explore_or_exploit = rng.binomial(1, delta)
		
		# exploration
		if explore_or_exploit == 1:
			# choose arm to pull uniformly at random
			chosen_arm = rng.integers(low=0, high=N)
			# observe loss--will be either 0 (no payout) or -1 (payout of $1)
			loss = 1-payouts[chosen_arm][t]
			simple_payout += payouts[chosen_arm][t]
			# update weight of chosen arm
			weights[chosen_arm] = weights[chosen_arm] * (1 - epsilon * loss)
			# update vector of selection probabilities
			W = sum(weights[i] for i in range(N))
			for i in range(N):
				x[i] = weights[i] / W

		# exploitation
		else:
			# choose arm to pull according to distribution x
			chosen_arm = rng.choice(N, p=x)
			# observe loss and update cumulative loss
			loss = 1-payouts[chosen_arm][t]
			simple_payout += payouts[chosen_arm][t]

	# display final vector of probabilities and cumulative reward
	print(x, simple_payout)

	# store cumulative reward for later analysis
	simple_payouts.append(simple_payout)


[0.08968791087135106, 0.09734149947313794, 0.18741788115434777, 0.24962349388643124, 0.37592921461473194] 565
[0.03658086390676398, 0.0764418296467561, 0.17336934686662245, 0.3204079201269198, 0.3932000394529377] 561
[0.04535869958584027, 0.08046545340755473, 0.1681460147443908, 0.2748345464822061, 0.4311952857800078] 569
[0.04123656398700556, 0.11956830764619183, 0.15286513853963266, 0.2943211325337129, 0.3920088572934569] 563
[0.05559529286880825, 0.0837256720473423, 0.14852771848963556, 0.26348529214865446, 0.4486660244455595] 561


In [6]:
# EXP3 Algorithm

# run algorithm K times
for k in range(K):
	# keep track of cumulative loss for EXP3
	exp3_payout = 0

	# initial vector of arm selection probabilities
	x = [(1/N) for i in range(N)]

	# pick an appropriate value for epsilon
	epsilon = np.sqrt(np.log(N) / (N*T))

	# algorithm
	for t in range(T):
		# placeholder vector y used to update x
		y = x.copy()
		# choose arm to pull according to distribution x
		chosen_arm = rng.choice(N, p=x)
		# observe loss and update cumulative loss
		loss = 1-payouts[chosen_arm][t]
		exp3_payout += payouts[chosen_arm][t]
		# calculate y-value for chosen arm
		y[chosen_arm] = x[chosen_arm] * np.exp(-epsilon * loss / x[chosen_arm])
		# update vector of selection probabilities
		ysum = sum(y[i] for i in range(N))
		for i in range(N):
			x[i] = y[i] / ysum

	# display final vector of probabilities and cumulative reward
	print(x, exp3_payout)

	# store cumulative reward for later analysis
	exp3_payouts.append(exp3_payout)


# calculate mean and variance of 100 cumulative rewards for each algorithm
# use statistics.mean(list) and statistics.variance(list)
print(statistics.mean(simple_payouts), statistics.variance(simple_payouts))

print(statistics.mean(exp3_payouts), statistics.variance(exp3_payouts))


[0.00022357711083561082, 0.00014903415255373197, 0.010490647674175776, 0.05973307263661681, 0.9294036684258181] 815
[5.161107193234813e-09, 0.004887074443199427, 3.182047185994903e-05, 0.07403404592410888, 0.9210470539997245] 809
[8.42599296094083e-05, 2.9108722108291046e-05, 3.4172315517497745e-13, 0.04590849961565624, 0.9539781317322844] 812
[1.0352481969478694e-06, 0.0004361541963693638, 0.014349511075547088, 0.007951997444586113, 0.9772613020353005] 818
[0.0007367228136716575, 1.163660182132731e-05, 0.006500025377004312, 0.11752331920006598, 0.8752282960074367] 802
563.8 11.2
815.4 55.15555555555555


In [16]:

# UCB Algorithm

# lists for storing numbers of pulls, empirical estimates of expected loss, lower confidence bounds for each arm
number_pulls = [0 for i in range(N)]
muhat = [0 for i in range(N)]
lcb = [0 for i in range(N)]

# variable for tracking total payout from arms chosen by UCB
ucb_payout = 0

# start by pulling each arm once and observing losses
for t in range(N):
	chosen_arm = t
	# update number of times arm has been pulled
	number_pulls[chosen_arm] += 1

	# observe arm payout and translate to loss
	loss = 1-payouts[chosen_arm][t]
	ucb_payout += payouts[chosen_arm][t]

	# initial empirical estimate of expected loss is just loss incurred on first pull
	muhat[chosen_arm] = loss

# now start pulling the arm with the smallest lower confidence bound at each round
for t in range(N,T):
	# calculate LCB for each arm
	for i in range(N):
		lcb[i] = muhat[i]-np.sqrt(3*np.log(t)/number_pulls[i])

	# choose arm with smallest LCB
	chosen_arm = lcb.index(min(lcb))

	# observe arm payout and translate to loss
	loss = 1-payouts[chosen_arm][t]
	ucb_payout += payouts[chosen_arm][t]

	# update number of pulls, empirical estimate of expected loss for chosen arm
	number_pulls[chosen_arm] += 1
	muhat[chosen_arm] = (muhat[chosen_arm]*(number_pulls[chosen_arm]-1)+loss)/number_pulls[chosen_arm]

print(muhat, ucb_payout)
print(statistics.mean(simple_payouts), statistics.variance(simple_payouts))

print(statistics.mean(exp3_payouts), statistics.variance(exp3_payouts))


[0.84, 0.6756756756756757, 0.4473684210526316, 0.30496453900709236, 0.087378640776699] 814
563.8 11.2
815.4 55.15555555555555


In [17]:
# Thompson Sampling Algorithm

# list for storing sampled probabilities of success for each arm
phat = [0 for i in range(N)]

# run algorithm K times
for k in range(K):
	
	# variable for tracking total payout from arms chosen by UCB
	ts_payout = 0

	# lists for storing alpha and beta parameters for each arm's beta distribution
	alpha = [1 for i in range(N)]
	beta = [1 for i in range(N)]

	# algorithm execution
	for t in range(T):
		# sample phat values from beta distributions
		for i in range(N):
			phat[i] = np.random.beta(alpha[i], beta[i],size=None)

		# choose arm with largest phat value
		chosen_arm = phat.index(max(phat))

		# observe arm payout
		reward = payouts[chosen_arm][t]
		ts_payout += reward

		# update beta distribution parameters for chosen arm
		if reward == 1:
			alpha[chosen_arm] += 1
		else:
			beta[chosen_arm] += 1

	ts_payouts.append(ts_payout)
	print(alpha, beta, ts_payout)

print(statistics.mean(ts_payouts), statistics.variance(ts_payouts))

[1, 1, 5, 10, 891] [4, 3, 5, 5, 85] 903
[2, 4, 5, 18, 874] [5, 5, 5, 7, 85] 898
[1, 3, 12, 45, 835] [3, 5, 9, 16, 81] 891
[1, 5, 21, 40, 827] [4, 5, 12, 14, 81] 889
[2, 2, 3, 1, 901] [5, 4, 4, 3, 85] 904
434.73333333333335 132638.20229885058
